GOAL: In TORONTO, if someone is looking to open a French Restaurant, where would you recommend that they open it?

Importing and installing all that can be necessary

In [1]:
import numpy as np 
import pandas as pd 
import types
from botocore.client import Config
import ibm_boto3
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
print('Libraries imported.')

Libraries imported.


In [2]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [3]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


altair-4.1.0         | 614 KB    | ##################################### | 1

Preparing the Foursquare setting

In [4]:
#Foursquare
CLIENT_ID = 'WS3IH3T0F5EXCVUTHBJFHJRJMJJHUV2P1WPOJPCPC4CQDOZL' # your Foursquare ID
CLIENT_SECRET = '0ASEX1X35VF41WVO0NH1XAXPTARBPG5LPFL4DUY35HCK2VOO' # your Foursquare Secret
VERSION = '20200422' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WS3IH3T0F5EXCVUTHBJFHJRJMJJHUV2P1WPOJPCPC4CQDOZL
CLIENT_SECRET:0ASEX1X35VF41WVO0NH1XAXPTARBPG5LPFL4DUY35HCK2VOO


We are setting in Toronto

In [5]:
address = 'Toronto, TO'
geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Preparing the map of Toronto

In [6]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [ ]:
#check
#map_toronto

Let's explore Toronto Neighborhood using Foursquare platform

In [7]:
LIMIT = 100 
radius = 500 
#create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WS3IH3T0F5EXCVUTHBJFHJRJMJJHUV2P1WPOJPCPC4CQDOZL&client_secret=0ASEX1X35VF41WVO0NH1XAXPTARBPG5LPFL4DUY35HCK2VOO&v=20200422&ll=43.6534817,-79.3839347&radius=500&limit=100'

In [8]:
results = requests.get(url).json()

Let's divide what we found in categories

In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
        
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

#filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

#filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

#clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980
3,Japango,Sushi Restaurant,43.655268,-79.385165
4,Indigo,Bookstore,43.653515,-79.380696


Now we define what are the most common categories for each Neighborhood

First, we prepare the Data of Toronto getting them from Wikipedia

In [11]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.drop(df[df['Borough']=='Not assigned'].index,inplace=True)
df.reindex

<bound method DataFrame.reindex of     Postal code           Borough  \
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
8           M9A         Etobicoke   
9           M1B       Scarborough   
11          M3B        North York   
12          M4B         East York   
13          M5B  Downtown Toronto   
14          M6B        North York   
17          M9B         Etobicoke   
18          M1C       Scarborough   
20          M3C        North York   
21          M4C         East York   
22          M5C  Downtown Toronto   
23          M6C              York   
26          M9C         Etobicoke   
27          M1E       Scarborough   
30          M4E      East Toronto   
31          M5E  Downtown Toronto   
32          M6E              York   
36          M1G       Scarborough   
39          M4G         East York   
40          M5G  Downtown Toronto   
41 

In [12]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3b5c2bdc58c94defb31bb5be8c5e2af7 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='huIpeVpPXm4bVzCZtfEBR68OfhcR5zhM0nI0r1THmmWy',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_3b5c2bdc58c94defb31bb5be8c5e2af7.get_object(Bucket='ibmcourseradatasciencemodule9caps-donotdelete-pr-pfzgdxkvbxscse',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data_0 = pd.read_csv(body)
df_data_0.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df_final = pd.merge(left=df, right=df_data_0, left_on='Postal code', right_on='Postal Code')
del df_final['Postal Code']
df_final.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


Then we are ready to get nearby venues

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    
Toronto_venues = getNearbyVenues(names=df_final['Neighborhood'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )
                                  
print(Toronto_venues.shape)
Toronto_venues.head()

Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [15]:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
#Toronto_grouped
Toronto_grouped.shape

(96, 271)

In [16]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2               Skating Rink  0.25
3                     Lounge  0.25
4                Yoga Studio  0.00


----Alderwood / Long Branch----
                venue  freq
0         Pizza Place  0.22
1  Athletics & Sports  0.11
2         Coffee Shop  0.11
3                 Pub  0.11
4      Sandwich Place  0.11


----Bathurst Manor / Wilson Heights / Downsview North----
            venue  freq
0     Coffee Shop  0.10
1            Bank  0.10
2     Pizza Place  0.05
3   Shopping Mall  0.05
4  Sandwich Place  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3  Japanese Restaurant  0.25
4  Moroccan Restaurant  0.00


----Bedford Park / Lawrence Manor East----
                     venue  freq
0              Coffee Shop  0.09
1               Restaurant  0.09
2           Sandwich P

          venue  freq
0         Trail   0.2
1         Field   0.2
2       Dog Run   0.2
3  Tennis Court   0.2
4  Hockey Arena   0.2


----India Bazaar / The Beaches West----
                  venue  freq
0        Sandwich Place  0.10
1  Fast Food Restaurant  0.10
2                  Park  0.10
3     Fish & Chips Shop  0.05
4    Italian Restaurant  0.05


----Kennedy Park / Ionview / East Birchmount Park----
                venue  freq
0         Coffee Shop  0.17
1       Train Station  0.17
2  Chinese Restaurant  0.17
3    Department Store  0.17
4         Bus Station  0.17


----Kensington Market / Chinatown / Grange Park----
                   venue  freq
0                   Café  0.09
1            Coffee Shop  0.07
2  Vietnamese Restaurant  0.05
3     Mexican Restaurant  0.05
4            Gaming Cafe  0.04


----Kingsview Village / St. Phillips / Martin Grove Gardens / Richview Gardens----
                venue  freq
0         Pizza Place  0.25
1      Sandwich Place  0.25
2   Mobile Ph

                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2              Brewery  0.05
3               Bakery  0.05
4  American Restaurant  0.05


----Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park----
              venue  freq
0       Coffee Shop  0.12
1               Pub  0.12
2              Bank  0.06
3  Sushi Restaurant  0.06
4        Restaurant  0.06


----The Annex / North Midtown / Yorkville----
                       venue  freq
0             Sandwich Place  0.14
1                       Café  0.14
2                Coffee Shop  0.09
3                   Pharmacy  0.05
4  Middle Eastern Restaurant  0.05


----The Beaches----
                        venue  freq
0                       Trail  0.25
1           Health Food Store  0.25
2                         Pub  0.25
3                 Yoga Studio  0.00
4  Modern European Restaurant  0.00


----The Danforth West / Riverdale----
                    venue  freq
0        Greek Restaur

In [19]:
nearby_venues['categories']

0                      Neighborhood
1                             Plaza
2                    Breakfast Spot
3                  Sushi Restaurant
4                         Bookstore
5                    Cosmetics Shop
6                   Bubble Tea Shop
7                     Shopping Mall
8                        Restaurant
9                       Coffee Shop
10                     Concert Hall
11                   Cosmetics Shop
12                       Art Museum
13             Fast Food Restaurant
14                   Clothing Store
15                 Ramen Restaurant
16    Vegetarian / Vegan Restaurant
17                          Theater
18                       Restaurant
19                   Clothing Store
20                            Hotel
21              Japanese Restaurant
22              American Restaurant
23                            Plaza
24           Furniture / Home Store
25                          Theater
26               Seafood Restaurant
27               Seafood Res

In [ ]:
We have a total of 70 categories

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 70

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,61th Most Common Venue,62th Most Common Venue,63th Most Common Venue,64th Most Common Venue,65th Most Common Venue,66th Most Common Venue,67th Most Common Venue,68th Most Common Venue,69th Most Common Venue,70th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,...,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Colombian Restaurant,College Rec Center
1,Alderwood / Long Branch,Pizza Place,Gym,Coffee Shop,Sandwich Place,Athletics & Sports,Pub,Skating Rink,Pharmacy,Department Store,...,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Food Truck,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court
2,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Bank,Sushi Restaurant,Pizza Place,Bridal Shop,Sandwich Place,Diner,Restaurant,Ice Cream Shop,...,Eastern European Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market
3,Bayview Village,Bank,Chinese Restaurant,Japanese Restaurant,Café,Women's Store,Doner Restaurant,Diner,Discount Store,Distribution Center,...,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Comfort Food Restaurant,College Rec Center
4,Bedford Park / Lawrence Manor East,Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Juice Bar,Pizza Place,Boutique,Comfort Food Restaurant,Liquor Store,...,Electronics Store,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant


Analyze the results

In [30]:
neighborhoods_venues_sorted.shape

(96, 71)

In [37]:
df70 = neighborhoods_venues_sorted['70th Most Common Venue'].str.find('French Restaurant')
df70.sum()

-95

There is a single Neighborhood row that as least common venue has a French Restaurant

In [46]:
DF_OK = neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['70th Most Common Venue'] =='French Restaurant']

In [47]:
DF_OK['Neighborhood']

80    The Danforth West / Riverdale
Name: Neighborhood, dtype: object

# The two best places in Toronto are Danforth West or Riverdale